In [1]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery

In [2]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [3]:
#Nombre de lignes totales dans le JSON : 1320229

In [4]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.events"

In [5]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [30]:
CHUNKSIZE = 100000

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=c.copy()
    ep.drop(['user_properties','event_properties'],axis=1,inplace=True)
    ep['user_id'] = ep['user_id'].astype(str)
    ep['id'] = (i*CHUNKSIZE) + index_init
    #ep.to_csv('../raw_data/events.csv',index=False)
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 100000 rows and 16 columns to marbotic_dataset.events
Loaded 200000 rows and 16 columns to marbotic_dataset.events
Loaded 300000 rows and 16 columns to marbotic_dataset.events
Loaded 400000 rows and 16 columns to marbotic_dataset.events
Loaded 500000 rows and 16 columns to marbotic_dataset.events
Loaded 600000 rows and 16 columns to marbotic_dataset.events
Loaded 700000 rows and 16 columns to marbotic_dataset.events
Loaded 800000 rows and 16 columns to marbotic_dataset.events
Loaded 900000 rows and 16 columns to marbotic_dataset.events
Loaded 1000000 rows and 16 columns to marbotic_dataset.events
Loaded 1100000 rows and 16 columns to marbotic_dataset.events
Loaded 1200000 rows and 16 columns to marbotic_dataset.events
Loaded 1300000 rows and 16 columns to marbotic_dataset.events
Loaded 1320229 rows and 16 columns to marbotic_dataset.events


In [18]:
#data = pd.read_csv('../raw_data/events.csv')
#data

,index,city,client_event_time,country,device_model,event_id,event_type,is_attribution_event,language,os_version,region,session_id,start_version,user_creation_time,user_id,id
0,0,Dallas,2022-03-01 00:57:25.903,United States,"iPad13,4",1110,[Activity] Good Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531.0,1
1,1,Dallas,2022-03-01 00:57:41.878,United States,"iPad13,4",1111,[Activity] Wrong Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531.0,2
2,2,Toronto,2022-03-01 00:57:28.175,Canada,"iPad12,1",1430,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882.0,3
3,3,Toronto,2022-03-01 00:57:28.177,Canada,"iPad12,1",1431,[Activity] Start,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882.0,4
4,4,Toronto,2022-03-01 00:57:38.278,Canada,"iPad12,1",1432,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,NaN,2022-03-05 03:50:32.017,United States,iPad 6,544,[Scene] Enter,False,English,15.3.1,Alabama,1646452197407,1.0.9,2022-03-03 12:22:10.906,70834.0,99996
99996,99996,NaN,2022-03-05 03:50:39.943,United States,iPad 6,545,[Activity] Start,False,English,15.3.1,Alabama,1646452197407,1.0.9,2022-03-03 12:22:10.906,70834.0,99997
99997,99997,NaN,2022-03-05 03:51:26.146,United States,iPad 6,546,[Activity] Wrong Answer,False,English,15.3.1,Alabama,1646452197407,1.0.9,2022-03-03 12:22:10.906,70834.0,99998
99998,99998,NaN,2022-03-05 03:53:40.727,United States,iPad 6,547,[Game] Pause,False,English,15.3.1,Alabama,1646452197407,1.0.9,2022-03-03 12:22:10.906,70834.0,99999
